In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import calendar
from matplotlib import pyplot as plt
import scipy.stats as stats
import glob

In [2]:
# 2019-2021年の3年分を1つのファイルに
# Time はDateTime型

In [3]:
# 全てを集約するデータファイル
df_king = pd.DataFrame()

In [4]:
# JEPX 
# spotはシステムプライスと約定量
# intraday は　平均価格と取引量
# これに価格差とTimeの列を加えた6つの列を追加


In [5]:
# JEPX2018-2021年度までのファイルをまとめて返す関数 
# spotはシステムプライスと約定量
# intraday は　平均価格と取引量
# これに価格差とTimeの列を加えた6つの列を追加
def Get_JEPX():

    # JEPX 
    # spotはシステムプライスと約定量
    # intraday は　平均価格と取引量
    # これに価格差とTimeの列を加えた6つの列を追加

    # 全年分のJEPX用の用の空のデータフレーム
    df_jepx_total = pd.DataFrame(columns=['DateTime','intra_price(円/kWh)','spot_price(円/kWh)','gap_price[intra-spot](円/kWh)','intra_volume（MWh/h）','spot_volume(kWh)'])

    for year in range(2018,2023):

    # year = 2021
            
        df_spot=pd.read_csv(f'../../卒論関連書類/データ/JEPX_data/spot_{year}.csv',encoding='Shift-JIS')
        df_intraday = pd.read_csv(f'../../卒論関連書類/データ/JEPX_data/im_trade_summary_{year}.csv',encoding='Shift-JIS')

        # 一年分のjepx用の空のデータフレーム
        df_jepx  = pd.DataFrame(columns=['DateTime','intra_price(円/kWh)','spot_price(円/kWh)','gap_price[intra-spot](円/kWh)','intra_volume（MWh/h）','spot_volume(kWh)'])
        # //日付をTimeStamp型で追加
        date_list=[]

        spot_dates = df_spot['年月日'].values
        # 時刻コードを時間に直す(0時0分ー23時30分)
        time_code =df_spot['時刻コード'].values
        time_code = (time_code-1)*30
        hour_list,min_list = time_code//60, time_code%60
        for i in range(len(spot_dates)):
            str_date = f'{spot_dates[i]}/{hour_list[i]}/{min_list[i]}'
            date = dt.datetime.strptime(str_date,'%Y/%m/%d/%H/%M')
            date_list.append(date)
        df_jepx['DateTime'] = date_list
        # //
        # 他のデータを代入
        df_jepx['spot_price(円/kWh)'] = df_spot['システムプライス(円/kWh)']
        df_jepx['intra_price(円/kWh)'] = df_intraday['平均（円/kWh）']
        df_jepx['gap_price[intra-spot](円/kWh)'] = df_intraday['平均（円/kWh）']-df_spot['システムプライス(円/kWh)']
        df_jepx['intra_volume（MWh/h）'] = df_intraday['約定量合計（MWh/h）']
        df_jepx['spot_volume(kWh)'] = df_spot['約定総量(kWh)']

        # 一年分を全体に合算
        df_jepx_total = pd.concat([df_jepx_total,df_jepx],axis=0)
    df_jepx_total = df_jepx_total.reset_index(drop=True)
    return(df_jepx_total)

# カンマ付きの数字をfloat型に変換する
def number_converter(array_number):
    str_numbers = array_number
    number_list =[]
    for str_number in str_numbers:
        if type(str_number)==str:

            str_number = str_number.replace(',','')
            float_number = float(str_number)
        else:
            float_number = float(str_number)
        number_list.append(float_number)
    return(number_list)

# 九州電力の想定値と実績地を返す関数
# 12月8日4:00の風力想定値のみ欠損
def Get_Kyuden_Plan():
    
    df_kyuden_total = pd.DataFrame(columns=['DateTime','九電太陽光想定(kWh)','九電太陽光実績(kWh)','九電風力想定(kWh)','九電風力実績(kWh)'])

    # 1-3が画翌年
    # 単位はkWh
    month_list =['4-6','7-9','10-12','1-3']
    sheet_name_list = ['太陽光想定','太陽光実績','風力想定','風力実績']
    col_names =np.array(list(range(49)),dtype='str')


    for year in range(2018,2023):
        if year==2022:
            month_list=month_list[:2]
        else:
            pass


        for month in month_list:
            for name_s in sheet_name_list[:]:

                df_read = pd.read_excel(f'../../卒論関連書類/データ/発電予測値/九州電力/td_{year}_{month}.xls',sheet_name=name_s,names=col_names)[3:]
                            # シート読み込みの初回にDatetime を作成する
                if name_s==sheet_name_list[0]:
                        # 一区切り分のデータをまとめるデータフレームの作成  
                    df = pd.DataFrame(columns=['DateTime','九電太陽光想定(kWh)','九電太陽光実績(kWh)','九電風力想定(kWh)','九電風力実績(kWh)'])

                    # ///読み取ったシートからTimeStampを制作する
                    # 日付の抽出
                    # Dates = df_read['0'].values
                    # 時刻のリストを作る
                    Times = np.array(list(range(0,48)))*30
                    hours,mins = Times//60,Times%60

                    datetime_list = []
                    #日付と時刻を合わせたリストを作る
                    for date in df_read['0']:
                        for i in range(48):

                            str_datetime = f'{date.year}/{date.month}/{date.day}/{hours[i]}/{mins[i]}'
                            datetime_list.append(dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H/%M'))
                    # ///
                    df['DateTime'] = datetime_list
                else:
                    pass

                # ここから上はシートの読み込み一回のみ一度のみの処理
                # ここから下はシートの数繰り返し
                values_list =[]
                for num in range(len(df_read)):
                    values = df_read[num:num+1].values[0][1:]
                    values_list.extend(values)
                if name_s=='太陽光想定':
                    df['九電太陽光想定(kWh)']=values_list
                elif name_s=='太陽光実績':
                    df['九電太陽光実績(kWh)']=values_list
                elif name_s=='風力想定':
                    df['九電風力想定(kWh)']=values_list
                elif name_s == '風力実績':
                    df['九電風力実績(kWh)']=values_list
                else:
                    print('エラー九電')
            
            df_kyuden_total = pd.concat([df_kyuden_total,df],axis=0)

    df_kyuden_total = df_kyuden_total.reset_index(drop=True)
    return(df_kyuden_total)


# 東電の想定値と実績地を返す関数
def Get_Toden_Plan():

    df_toden_plan_king = pd.DataFrame(columns=['DateTime','東電太陽光想定(kWh)','東電太陽光実績(kWh)','東電風力想定(kWh)','東電風力実績(kWh)'])

    for year in range(2018,2023):
        df_toden_plan = pd.read_csv(f'../../卒論関連書類/データ/発電予測値/東京電力/fit-{year}.csv',encoding='shift-jis')
        df_toden_plan_total = pd.DataFrame(columns=['DateTime','東電太陽光想定(kWh)','東電太陽光実績(kWh)','東電風力想定(kWh)','東電風力実績(kWh)'])

        # timestampの列を作成
        date_list =  df_toden_plan['DATE'].values
        time_list = df_toden_plan['TIME'].values
        str_datetime_list = date_list+'/'+time_list

        datetime_list = []
        for str_datetime in str_datetime_list:
            datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M')
            datetime_list.append(datetime)
        # 値の代入
        df_toden_plan_total['DateTime'] = datetime_list
        df_toden_plan_total['東電太陽光想定(kWh)'] = df_toden_plan['太陽光想定(kWh)']
        df_toden_plan_total['東電太陽光実績(kWh)'] = df_toden_plan['太陽光実績(kWh)']
        df_toden_plan_total['東電風力想定(kWh)'] = df_toden_plan['風力想定(kWh)']
        df_toden_plan_total['東電風力実績(kWh)'] = df_toden_plan['風力実績(kWh)']

        # 結合
        df_toden_plan_king = pd.concat([df_toden_plan_king,df_toden_plan_total],axis=0)
    df_toden_plan_king = df_toden_plan_king.reset_index(drop=True)
    return(df_toden_plan_king)

# 関電の想定値と実績地を返す関数
def Get_Kanden_Plan():

    df_kanden_plan_king = pd.DataFrame(columns=['DateTime','関電太陽光想定(kWh)','関電太陽光実績(kWh)','関電風力想定(kWh)','関電風力実績(kWh)'])

    for year in range(2018,2023):
        df_kanden_plan =pd.read_csv(f'../../卒論関連書類/データ/発電予測値/関西電力/{year}fit1_soutei_jisseki.csv',encoding='shift-jis')
        df_kanden_plan_total = pd.DataFrame(columns=['DateTime','関電太陽光想定(kWh)','関電太陽光実績(kWh)','関電風力想定(kWh)','関電風力実績(kWh)'])

        str_datetime_list = df_kanden_plan['DATE_TIME'].values
        datetime_list=[]
        for str_datetime in str_datetime_list:
            datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d %H:%M')
            datetime_list.append(datetime)
        # 値の代入
        df_kanden_plan_total['DateTime'] = datetime_list
        df_kanden_plan_total['関電太陽光想定(kWh)'] = number_converter(df_kanden_plan['FIT1_太陽光想定値［kWh］'].values)
        df_kanden_plan_total['関電太陽光実績(kWh)'] = number_converter(df_kanden_plan['FIT1_太陽光実績値［kWh］'].values)
        df_kanden_plan_total['関電風力想定(kWh)'] = number_converter(df_kanden_plan['FIT1_風力想定値［kWh］'].values)
        df_kanden_plan_total['関電風力実績(kWh)'] = number_converter(df_kanden_plan['FIT1_風力実績値［kWh］'].values)

        # 結合
        df_kanden_plan_king = pd.concat([df_kanden_plan_king,df_kanden_plan_total],axis=0)
    df_kanden_plan_king = df_kanden_plan_king.reset_index(drop=True)
    return(df_kanden_plan_king)


#中国電力の想定値と実績値を返す関数
 
def Get_Chugoku_Plan():
    columns = ['DATE','TIME','中国太陽光想定(kWh)','中国風力想定(kWh)','中国太陽光実績(kWh)','中国風力実績(kWh)']
    # ファイルを全て読み込み
    df_chugoku_plan_2020_ = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/fit_tokurei1.csv',encoding='shift-jis',names=columns)[3:]
    df_chugoku_plan_2018 = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/kako-2018.csv',encoding='shift-jis',names=columns)[3:].dropna()
    df_chugoku_plan_2019 = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/kako-2019.csv',encoding='shift-jis',names=columns)[3:].dropna()
    # データを結合
    df_total_chugoku_plan =pd.concat([df_chugoku_plan_2018,df_chugoku_plan_2019,df_chugoku_plan_2020_],axis=0).reset_index(drop=True)
    # //timestampを作成
    datetime_list = []
    date_list, time_list = df_total_chugoku_plan['DATE'].values, df_total_chugoku_plan['TIME'].values

    for i in range(len(date_list)):
        str_datetime = date_list[i]+'/'+time_list[i]
        datetime_list.append(dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M'))
    # //
    df_total_chugoku_plan['DateTime'] = datetime_list
    df_total_chugoku_plan = df_total_chugoku_plan.drop(columns=['DATE','TIME'])
    # 左側にDateTimeを持ってきた
    df_total_chugoku_plan =df_total_chugoku_plan.reindex(columns=['DateTime','中国太陽光想定(kWh)','中国風力想定(kWh)','中国太陽光実績(kWh)','中国風力実績(kWh)']) 
    return(df_total_chugoku_plan)

# 予備率を地域ごとの列にして返す関数
def Get_yobiritu():
    # 広域予備率はエリアごとでまとめる
    # 沖縄だけは省く
    df_yobiritu = pd.read_csv('../../卒論関連書類/データ/広域予備率/yobiritsu.csv')
    datetime_list = []
    df_yobiritu=df_yobiritu[:]
    str_datelist = df_yobiritu['Date'].values
    str_timelist = df_yobiritu['Time'].values
    for i in range(len(df_yobiritu)):
        # 00:30始まり24:00終わりになっているため、00:00始まり23:30終わりに直す
        str_time =str_timelist[i]
        change_into_minute = int(str_time.split(':')[0])*60 + int(str_time.split(':')[1])-30
        new_hour = change_into_minute//60
        new_min = change_into_minute%60

        str_datetime = str_datelist[i]+'/'+str(new_hour) +':'+ str(new_min)
        Datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M')
        datetime_list.append(Datetime)
    df_yobiritu['DateTime'] = datetime_list
    # まとめるデータフレーム作成
    df_yobiritu_king = pd.DataFrame(columns=['DateTime','北海道予備率(%)','東北予備率(%)','東京予備率(%)','中部予備率(%)','北陸予備率(%)','関西予備率(%)','中国予備率(%)','四国予備率(%)','九州予備率(%)'])
    df_yobiritu_king['DateTime'] = df_yobiritu['DateTime']
    # 日付が重複しているから削除
    df_yobiritu_king = df_yobiritu_king.drop_duplicates('DateTime')
    area_list = ['北海道','東北','東京','中部','北陸','関西','中国','四国','九州']
    for area_name in area_list:

        df_yobiritu_king[f'{area_name}予備率(%)']  =df_yobiritu[df_yobiritu['エリア名']==area_name]['広域予備率(%)'].values
    df_yobiritu_king = df_yobiritu_king.reset_index(drop=True)

    return(df_yobiritu_king)

# 北電の想定値と実績地を返す関数
def Get_Hokuden_Plan():

    columns =['Date','Time','NAN1','北電太陽光想定(MW)','北電風力想定(MW)','NAN2','北電太陽光実績(MW)','北電風力実績(MW)']

    df_hokuden_jan =pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_jan_mar.csv',encoding='cp932',names=columns)[3:]
    df_hokuden_apr = pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_apr_june.csv',encoding='cp932',names=columns)[3:]
    df_hokuden_july = pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_july_sept.csv',encoding='cp932',names=columns)[3:]
    df_hokuden_oct =pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_oct_dec.csv',encoding='cp932',names=columns)[3:]
    # //まとめるファイルの作成
    df_hokuden_plan = pd.concat([df_hokuden_jan,df_hokuden_apr,df_hokuden_july,df_hokuden_oct],axis=0).reset_index(drop=True)
    #//
    # ///TimeStampの作成
    datetime_list =[]
    str_datelist = df_hokuden_plan['Date'].values
    str_timelist = df_hokuden_plan['Time'].values
    for i in range(len(df_hokuden_plan)):
        str_datetime = str_datelist[i]+'/'+str_timelist[i]
        Datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M')
        datetime_list.append(Datetime)
    df_hokuden_plan['DateTime'] = datetime_list
    # ///
    df_hokuden_plan= df_hokuden_plan.drop(columns=['Date','Time','NAN1','NAN2'])
    # 整えた
    df_hokuden_plan= df_hokuden_plan.reindex(columns=['DateTime','北電太陽光想定(MW)', '北電風力想定(MW)', '北電太陽光実績(MW)', '北電風力実績(MW)'])
    return(df_hokuden_plan)
							

In [11]:
# JEPXのデータ
df_jepx_total = Get_JEPX()
df_kyuden_plan = Get_Kyuden_Plan()
df_toden_plan = Get_Toden_Plan()
df_kanden_plan =Get_Kanden_Plan()
df_chugoku_plan = Get_Chugoku_Plan()
df_yobiritu = Get_yobiritu()
df_hokuden_plan = Get_Hokuden_Plan()


In [17]:
df_stop = pd.read_csv('../../卒論関連書類/データ/電源停止データ/電源停止データ整理済み.csv')

In [20]:
df_stop['DateTime'] = pd.to_datetime(df_stop['DateTime'])

In [121]:
df_yobiritu

,DateTime,北海道予備率(%),東北予備率(%),東京予備率(%),中部予備率(%),北陸予備率(%),関西予備率(%),中国予備率(%),四国予備率(%),九州予備率(%)
0,2022-03-21 00:00:00,21.35,21.35,21.35,21.35,21.35,21.35,21.35,21.35,21.35
1,2022-03-21 00:30:00,22.82,22.82,22.82,22.82,22.82,22.82,22.82,22.82,22.82
2,2022-03-21 01:00:00,22.48,22.48,22.48,22.77,22.77,22.77,22.77,22.77,22.77
3,2022-03-21 01:30:00,22.26,22.26,22.26,22.26,22.26,22.26,22.26,22.26,22.26
4,2022-03-21 02:00:00,21.83,21.83,21.83,21.83,21.83,21.83,21.83,21.83,21.83
...,...,...,...,...,...,...,...,...,...,...
13723,2022-12-31 21:30:00,26.43,36.82,36.82,27.10,27.10,27.10,27.10,27.10,27.10
13724,2022-12-31 22:00:00,27.55,39.19,39.19,28.01,28.01,28.01,28.01,28.01,28.01
13725,2022-12-31 22:30:00,30.03,40.96,40.96,31.06,31.06,31.06,31.06,31.06,31.06
13726,2022-12-31 23:00:00,21.80,40.62,40.62,29.03,29.03,29.03,29.03,29.03,29.03


In [21]:
# 集約させて保存
# バージョン分けされているから実行分だけファイルが生成されるため注意
# 全てを集約するデータファイル
df_king = pd.DataFrame()
# JEPXを統合
df_king = pd.concat([df_king,df_jepx_total],axis=1)
# 電力の想定値と実績地
df_king =pd.merge(df_king,df_hokuden_plan,on='DateTime',how='outer')
df_king = pd.merge(df_king,df_kyuden_plan,on='DateTime',how='outer')
df_king = pd.merge(df_king,df_toden_plan,on='DateTime',how='outer')
df_king = pd.merge(df_king,df_kanden_plan,on='DateTime',how='outer')
df_king =pd.merge(df_king,df_chugoku_plan,on='DateTime',how='outer')

# 予備率
df_king =pd.merge(df_king,df_yobiritu,on='DateTime',how='outer')

# 停止情報
df_king = pd.merge(df_king,df_stop,on='DateTime',how='outer')

files = glob.glob('../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)
df_king.to_csv(f'../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',index=False)

In [193]:
df_king.dtypes

DateTime                        datetime64[ns]
intra_price(円/kWh)                     float64
spot_price(円/kWh)                      float64
gap_price[intra-spot](円/kWh)           float64
intra_volume（MWh/h）                    float64
spot_volume(kWh)                        object
北電太陽光想定(MW)                             object
北電風力想定(MW)                              object
北電太陽光実績(MW)                             object
北電風力実績(MW)                              object
九電太陽想定値(kWh)                           float64
九電太陽実績値(kWh)                           float64
九電風力想定値(kWh)                           float64
九電風力実績値(kWh)                           float64
東電太陽光想定(kWh)                            object
東電太陽光実績(kWh)                            object
東電風力想定(kWh)                             object
東電風力実績(kWh)                             object
関電太陽光想定(kWh)                           float64
関電太陽光実績(kWh)                           float64
関電風力想定(kWh)                            float64
関電風力実績(kWh)  

In [287]:
df_king = pd.DataFrame()
df_king = pd.concat([df_king,df_jepx_total],axis=1)
df_king = pd.merge(df_king,df_kyuden_plan,on='DateTime')
df_king = pd.merge(df_king,df_toden_plan,on='DateTime')
df_king = pd.merge(df_king,df_kanden_plan,on='DateTime')
df_king =pd.merge(df_king,df_chugoku_plan,on='DateTime')
df_king

,DateTime,intra_price(円/kWh),spot_price(円/kWh),gap_price[intra-spot](円/kWh),intra_volume（MWh/h）,spot_volume(kWh),九電太陽想定値(kWh),九電太陽実績値(kWh),九電風力想定値(kWh),九電風力実績値(kWh),...,東電風力想定(kWh),東電風力実績(kWh),関電太陽光想定(kWh),関電太陽光実績(kWh),関電風力想定(kWh),関電風力実績(kWh),中国太陽光想定(kWh),中国風力想定(kWh),中国太陽光実績(kWh),中国風力実績(kWh)
0,2018-04-01 00:00:00,7.62,7.30,0.32,58.8,6098500,0.0,0.0,5889.905000,8909.100000,...,41386,50919,0,0,18118,21411,0,39041,0,38993
1,2018-04-01 00:30:00,7.52,6.93,0.59,54.0,6225500,0.0,0.0,5642.430000,8909.100000,...,41615,50733,0,0,18118,22064,0,41624,0,40253
2,2018-04-01 01:00:00,7.50,6.78,0.72,57.3,6356500,0.0,0.0,5642.430000,6929.300000,...,41793,51667,0,0,18118,21297,0,43205,0,37977
3,2018-04-01 01:30:00,7.56,6.84,0.72,63.0,6439500,0.0,0.0,5444.450000,5939.400000,...,42116,57049,0,0,18118,19419,0,43721,0,39237
4,2018-04-01 02:00:00,6.89,6.92,-0.03,61.6,6439000,0.0,0.0,5295.965000,5939.400000,...,42244,55349,0,0,18118,14621,0,43721,0,37666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70123,2022-03-31 21:30:00,22.13,23.40,-1.27,1116.4,17418300,0.0,0.0,75589.876734,80915.101876,...,79325,67314,0,0,"17,549","45,310",0,26596,0,53391
70124,2022-03-31 22:00:00,24.66,28.34,-3.68,995.2,16727350,0.0,0.0,78357.541014,82754.081464,...,81166,72550,0,0,"18,966","52,083",0,27563,0,61020
70125,2022-03-31 22:30:00,17.31,23.96,-6.65,797.1,16534600,0.0,0.0,76204.555661,85512.550846,...,85104,71513,0,0,"20,660","60,271",0,28425,0,60784
70126,2022-03-31 23:00:00,15.81,22.80,-6.99,649.0,16452950,0.0,0.0,76632.118415,89190.510023,...,89412,66491,0,0,"22,572","61,016",0,28154,0,61375


In [173]:
df_king['関電太陽光実績(kWh)']

0          0
1          0
2          0
3          0
4          0
        ... 
83672    NaN
83673    NaN
83674    NaN
83675    NaN
83676    NaN
Name: 関電太陽光実績(kWh), Length: 83677, dtype: object

In [134]:
columns =['Date','Time','NAN1','北電太陽光想定(MW)','北電風力想定(MW)','NAN2','北電太陽光実績(MW)','北電風力実績(MW)']

df_hokuden_jan =pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_jan_mar.csv',encoding='cp932',names=columns)[3:]
df_hokuden_apr = pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_apr_june.csv',encoding='cp932',names=columns)[3:]
df_hokuden_july = pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_july_sept.csv',encoding='cp932',names=columns)[3:]
df_hokuden_oct =pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_oct_dec.csv',encoding='cp932',names=columns)[3:]



In [144]:
df_hokuden_plan = pd.concat([df_hokuden_jan,df_hokuden_apr,df_hokuden_july,df_hokuden_oct],axis=0).reset_index(drop=True)

In [162]:
def Get_Hokuden_Plan():

    columns =['Date','Time','NAN1','北電太陽光想定(MW)','北電風力想定(MW)','NAN2','北電太陽光実績(MW)','北電風力実績(MW)']

    df_hokuden_jan =pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_jan_mar.csv',encoding='cp932',names=columns)[3:]
    df_hokuden_apr = pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_apr_june.csv',encoding='cp932',names=columns)[3:]
    df_hokuden_july = pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_july_sept.csv',encoding='cp932',names=columns)[3:]
    df_hokuden_oct =pd.read_csv('../../卒論関連書類/データ/発電予測値/北海道電力/expect_oct_dec.csv',encoding='cp932',names=columns)[3:]
    # //まとめるファイルの作成
    df_hokuden_plan = pd.concat([df_hokuden_jan,df_hokuden_apr,df_hokuden_july,df_hokuden_oct],axis=0).reset_index(drop=True)
    #//
    # ///TimeStampの作成
    datetime_list =[]
    str_datelist = df_hokuden_plan['Date'].values
    str_timelist = df_hokuden_plan['Time'].values
    for i in range(len(df_hokuden_plan)):
        str_datetime = str_datelist[i]+'/'+str_timelist[i]
        Datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M')
        datetime_list.append(Datetime)
    df_hokuden_plan['DateTime'] = datetime_list
    # ///
    df_hokuden_plan= df_hokuden_plan.drop(columns=['Date','Time','NAN1','NAN2'])
    # 整えた
    df_hokuden_plan= df_hokuden_plan.reindex(columns=['DateTime','北電太陽光想定(MW)', '北電風力想定(MW)', '北電太陽光実績(MW)', '北電風力実績(MW)'])
    return(df_hokuden_plan)



In [178]:
str_numbers = df_kanden_plan['関電太陽光想定(kWh)'].values

In [187]:
# カンマ付きの数字をfloat型に変換する
def number_converter(array_number):

    number_list =[]
    for str_number in str_numbers:
        if type(str_number)==str:

            str_number = str_number.replace(',','')
            float_number = float(str_number)
        else:
            float_number = float(str_number)
        number_list.append(float_number)
    return(number_list)

In [186]:
number_list

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 102.0,
 72120.0,
 234941.0,
 436212.0,
 598895.0,
 791065.0,
 987436.0,
 1149807.0,
 1288125.0,
 1400305.0,
 1476276.0,
 1519906.0,
 1512830.0,
 1497409.0,
 1452104.0,
 1361992.0,
 1241551.0,
 1123358.0,
 983227.0,
 800474.0,
 624542.0,
 438044.0,
 287771.0,
 139376.0,
 48219.0,
 807.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 243.0,
 71572.0,
 210470.0,
 367083.0,
 481584.0,
 638208.0,
 842657.0,
 1005184.0,
 1134849.0,
 1207245.0,
 1276793.0,
 1308449.0,
 1315182.0,
 1302361.0,
 1311327.0,
 1283363.0,
 1220024.0,
 1135958.0,
 1036901.0,
 894277.0,
 742374.0,
 570785.0,
 415254.0,
 231226.0,
 84819.0,
 1344.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 453.0,
 64195.0,
 177874.0,
 337733.0,
 478303.0,
 642916.0,
 823507.0,
 1002736.0,

In [156]:
df_hokuden_plan= df_hokuden_plan.drop(columns=['Date','Time','NAN1','NAN2'])
df_hokuden_plan= df_hokuden_plan.reindex(columns=['DateTime','北電太陽光想定(MW)', '北電風力想定(MW)', '北電太陽光実績(MW)', '北電風力実績(MW)'])

KeyError: "['Date', 'Time', 'NAN1', 'NAN2'] not found in axis"

In [161]:
df_hokuden_plan= df_hokuden_plan.reindex(columns=['DateTime','北電太陽光想定(MW)', '北電風力想定(MW)', '北電太陽光実績(MW)', '北電風力実績(MW)'])

In [160]:
df_hokuden_plan

,北電太陽光想定(MW),北電風力想定(MW),北電太陽光実績(MW),北電風力実績(MW),DateTime
0,0,170.2,0,142.2,2022-01-01 00:00:00
1,0,170.4,0,136.8,2022-01-01 00:30:00
2,0,170.2,0,141.2,2022-01-01 01:00:00
3,0,170.9,0,142.5,2022-01-01 01:30:00
4,0,172.5,0,138.1,2022-01-01 02:00:00
...,...,...,...,...,...
17515,0,131.5,0,129.3,2021-12-31 21:30:00
17516,0,135.4,0,123.7,2021-12-31 22:00:00
17517,0,136.9,0,122.3,2021-12-31 22:30:00
17518,0,136.2,0,124.6,2021-12-31 23:00:00


In [112]:

def Get_yobiritu():
    # 広域予備率はエリアごとでまとめる
    # 沖縄だけは省く
    df_yobiritu = pd.read_csv('../../卒論関連書類/データ/広域予備率/yobiritsu.csv')
    datetime_list = []
    df_yobiritu=df_yobiritu[:]
    str_datelist = df_yobiritu['Date'].values
    str_timelist = df_yobiritu['Time'].values
    for i in range(len(df_yobiritu)):
        # 00:30始まり24:00終わりになっているため、00:00始まり23:30終わりに直す
        str_time =str_timelist[i]
        change_into_minute = int(str_time.split(':')[0])*60 + int(str_time.split(':')[1])-30
        new_hour = change_into_minute//60
        new_min = change_into_minute%60

        str_datetime = str_datelist[i]+'/'+str(new_hour) +':'+ str(new_min)
        Datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M')
        datetime_list.append(Datetime)
    df_yobiritu['DateTime'] = datetime_list
    # まとめるデータフレーム作成
    df_yobiritu_king = pd.DataFrame(columns=['Datetime','北海道予備率(%)','東北予備率(%)','東京予備率(%)','中部予備率(%)','北陸予備率(%)','関西予備率(%)','中国予備率(%)','四国予備率(%)','九州予備率(%)'])
    df_yobiritu_king['Datetime'] = df_yobiritu['DateTime']
    # 日付が重複しているから削除
    df_yobiritu_king = df_yobiritu_king.drop_duplicates('Datetime')
    area_list = ['北海道','東北','東京','中部','北陸','関西','中国','四国','九州']
    for area_name in area_list:

        df_yobiritu_king[f'{area_name}予備率(%)']  =df_yobiritu[df_yobiritu['エリア名']==area_name]['広域予備率(%)'].values
    df_yobiritu_king = df_yobiritu_king.reset_index(drop=True)

    return(df_yobiritu_king)
        

In [113]:
Get_yobiritu()

,Datetime,北海道予備率(%),東北予備率(%),東京予備率(%),中部予備率(%),北陸予備率(%),関西予備率(%),中国予備率(%),四国予備率(%),九州予備率(%)
0,2022-03-21 00:00:00,21.35,21.35,21.35,21.35,21.35,21.35,21.35,21.35,21.35
1,2022-03-21 00:30:00,22.82,22.82,22.82,22.82,22.82,22.82,22.82,22.82,22.82
2,2022-03-21 01:00:00,22.48,22.48,22.48,22.77,22.77,22.77,22.77,22.77,22.77
3,2022-03-21 01:30:00,22.26,22.26,22.26,22.26,22.26,22.26,22.26,22.26,22.26
4,2022-03-21 02:00:00,21.83,21.83,21.83,21.83,21.83,21.83,21.83,21.83,21.83
...,...,...,...,...,...,...,...,...,...,...
13723,2022-12-31 21:30:00,26.43,36.82,36.82,27.10,27.10,27.10,27.10,27.10,27.10
13724,2022-12-31 22:00:00,27.55,39.19,39.19,28.01,28.01,28.01,28.01,28.01,28.01
13725,2022-12-31 22:30:00,30.03,40.96,40.96,31.06,31.06,31.06,31.06,31.06,31.06
13726,2022-12-31 23:00:00,21.80,40.62,40.62,29.03,29.03,29.03,29.03,29.03,29.03


In [291]:
def Get_data_yobiritsu():

    month_list = list(range(1,13))

    # 広域予備率のデータをまとめる

    occto_columns = ['Date', 'Time', 'ブロックNo', 'エリア名', '広域ブロック需要(MW)', '広域ブロック供給力(MW)',
        '広域ブロック予備力(MW)', '広域予備率(%)', '広域使用率(%)', 'エリア需要(MW)', 'エリア供給力(MW)',
        'エリア予備力(MW)']
    df_occto = pd.DataFrame(columns=occto_columns)
    for i in month_list:
        month_range = calendar.monthrange(2022,i)[1]
        month_name= str(i)
        
        if len(month_name)==1:
            month_name = '0'+month_name
        elif len(month_name)==2:
            pass
        else:
            # print("error")
            break
    # ダウンロード先ののURL
        read_url = f'https://web-kohyo.occto.or.jp/kks-web-public/download/downloadCsv?jhSybt=02&tgtYmdFrom=2022%2F{month_name}%2F01&tgtYmdTo=2022%2F{month_name}%2F{month_range}'
        
        try:
            df_read= pd.read_csv(read_url)
            df_read = df_read.reset_index(drop=False)
            df_read.columns=occto_columns
            df_read = df_read[1:]
            df_occto = pd.concat([df_occto,df_read],axis=0)
            
        except:
            pass
    df_occto = df_occto.reset_index(drop=True)

    df_occto.to_csv('../../卒論関連書類/データ/広域予備率/yobiritsu.csv')


In [292]:
Get_data_yobiritsu()

In [51]:
df_yobiritu = pd.read_csv('../../卒論関連書類/データ/広域予備率/yobiritsu.csv')

In [109]:
# 広域予備率はエリアごとでまとめる
# 沖縄だけは省く
df_yobiritu = pd.read_csv('../../卒論関連書類/データ/広域予備率/yobiritsu.csv')
datetime_list = []
df_yobiritu=df_yobiritu[:]
str_datelist = df_yobiritu['Date'].values
str_timelist = df_yobiritu['Time'].values
for i in range(len(df_yobiritu)):
    # 00:30始まり24:00終わりになっているため、00:00始まり23:30終わりに直す
    str_time =str_timelist[i]
    change_into_minute = int(str_time.split(':')[0])*60 + int(str_time.split(':')[1])-30
    new_hour = change_into_minute//60
    new_min = change_into_minute%60

    str_datetime = str_datelist[i]+'/'+str(new_hour) +':'+ str(new_min)
    Datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M')
    datetime_list.append(Datetime)
df_yobiritu['DateTime'] = datetime_list
# まとめるデータフレーム作成
df_yobiritu_king = pd.DataFrame(columns=['Datetime','北海道予備率(%)','東北予備率(%)','東京予備率(%)','中部予備率(%)','北陸予備率(%)','関西予備率(%)','中国予備率(%)','四国予備率(%)','九州予備率(%)'])
df_yobiritu_king['Datetime'] = df_yobiritu['DateTime']
# 日付が重複しているから削除
df_yobiritu_king = df_yobiritu_king.drop_duplicates('Datetime')
area_list = ['北海道','東北','東京','中部','北陸','関西','中国','四国','九州']
for area_name in area_list:

    df_yobiritu_king[f'{area_name}予備率(%)']  =df_yobiritu[df_yobiritu['エリア名']==area_name]['広域予備率(%)'].values
df_yobiritu_king = df_yobiritu_king.reset_index(drop=True)


In [105]:
df_yobiritu_king = pd.DataFrame(columns=['Datetime','北海道予備率(%)','東北予備率(%)','東京予備率(%)','中部予備率(%)','北陸予備率(%)','関西予備率(%)','中国予備率(%)','四国予備率(%)','九州予備率(%)'])
df_yobiritu_king['Datetime'] = df_yobiritu['DateTime']
# 日付が重複しているから削除
df_yobiritu_king = df_yobiritu_king.drop_duplicates('Datetime')
area_list = ['北海道','東北','東京','中部','北陸','関西','中国','四国','九州']
for area_name in area_list:

    df_yobiritu_king[f'{area_name}予備率(%)']  =df_yobiritu[df_yobiritu['エリア名']==area_name]['広域予備率(%)'].values

In [110]:
df_yobiritu_king

,Datetime,北海道予備率(%),東北予備率(%),東京予備率(%),中部予備率(%),北陸予備率(%),関西予備率(%),中国予備率(%),四国予備率(%),九州予備率(%)
0,2022-03-21 00:00:00,21.35,21.35,21.35,21.35,21.35,21.35,21.35,21.35,21.35
1,2022-03-21 00:30:00,22.82,22.82,22.82,22.82,22.82,22.82,22.82,22.82,22.82
2,2022-03-21 01:00:00,22.48,22.48,22.48,22.77,22.77,22.77,22.77,22.77,22.77
3,2022-03-21 01:30:00,22.26,22.26,22.26,22.26,22.26,22.26,22.26,22.26,22.26
4,2022-03-21 02:00:00,21.83,21.83,21.83,21.83,21.83,21.83,21.83,21.83,21.83
...,...,...,...,...,...,...,...,...,...,...
13723,2022-12-31 21:30:00,26.43,36.82,36.82,27.10,27.10,27.10,27.10,27.10,27.10
13724,2022-12-31 22:00:00,27.55,39.19,39.19,28.01,28.01,28.01,28.01,28.01,28.01
13725,2022-12-31 22:30:00,30.03,40.96,40.96,31.06,31.06,31.06,31.06,31.06,31.06
13726,2022-12-31 23:00:00,21.80,40.62,40.62,29.03,29.03,29.03,29.03,29.03,29.03


In [96]:
len(df_yobiritu_king)

137280

In [101]:
df_yobiritu_king

,Datetime,北海道予備率(%),東北予備率(%),東京予備率(%),中部予備率(%),北陸予備率(%),関西予備率(%),中国予備率(%),四国予備率(%),九州予備率(%)
0,2022-03-21 00:00:00,21.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2022-03-21 00:30:00,22.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2022-03-21 01:00:00,22.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,2022-03-21 01:30:00,22.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2022-03-21 02:00:00,21.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
137230,2022-12-31 21:30:00,26.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137240,2022-12-31 22:00:00,27.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137250,2022-12-31 22:30:00,30.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137260,2022-12-31 23:00:00,21.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df_yobiritu[df_yobiritu['エリア名']==area_name]['広域予備率(%)']

0         21.35
10        22.82
20        22.48
30        22.26
40        21.83
          ...  
137230    26.43
137240    27.55
137250    30.03
137260    21.80
137270    21.27
Name: 広域予備率(%), Length: 13728, dtype: float64

In [67]:
# 00:30始まり24:00終わりになっているため、00:00始まり23:30終わりに直す
str_time =str_timelist[150]

In [77]:
datetime_list[150]

datetime.datetime(2022, 3, 21, 7, 30)

In [71]:
# 00:30始まり24:00終わりになっているため、00:00始まり23:30終わりに直す
str_time =str_timelist[i]
change_into_minute = int(str_time.split(':')[0])*60 + int(str_time.split(':')[1])-30
new_hour = change_into_minute//60
new_min = change_into_minute%60



In [72]:
new_hour

7

In [48]:
df_yobiritu

,Unnamed: 0,Date,Time,ブロックNo,エリア名,広域ブロック需要(MW),広域ブロック供給力(MW),広域ブロック予備力(MW),広域予備率(%),広域使用率(%),エリア需要(MW),エリア供給力(MW),エリア予備力(MW)
0,0,2022/03/21,00:30,1,北海道,82893.892,100593.054,17699.162,21.35,82.41,3524.000,4055.000,531.000
1,1,2022/03/21,00:30,1,東北,82893.892,100593.054,17699.162,21.35,82.41,8799.696,10747.652,1947.956
2,2,2022/03/21,00:30,1,東京,82893.892,100593.054,17699.162,21.35,82.41,25576.000,31802.000,6226.000
3,3,2022/03/21,00:30,1,中部,82893.892,100593.054,17699.162,21.35,82.41,12054.152,13794.798,1740.646
4,4,2022/03/21,00:30,1,北陸,82893.892,100593.054,17699.162,21.35,82.41,3054.000,3566.000,512.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,2022/03/21,05:00,1,関西,86789.458,104507.790,17718.332,20.42,83.05,13746.600,17934.600,4188.000
96,96,2022/03/21,05:00,1,中国,86789.458,104507.790,17718.332,20.42,83.05,6710.000,7354.160,644.160
97,97,2022/03/21,05:00,1,四国,86789.458,104507.790,17718.332,20.42,83.05,2986.194,4014.924,1028.730
98,98,2022/03/21,05:00,1,九州,86789.458,104507.790,17718.332,20.42,83.05,8931.000,10096.000,1165.000


In [285]:
df_king = pd.DataFrame()
# JEPXを統合
df_king = pd.concat([df_king,df_jepx_total],axis=1)
df_king = pd.merge(df_king,df_kyuden_plan,on='DateTime',how='outer')
df_king = pd.merge(df_king,df_toden_plan,on='DateTime',how='outer')
df_king = pd.merge(df_king,df_kanden_plan,on='DateTime',how='outer')
df_king =pd.merge(df_king,df_chugoku_plan,on='DateTime',how='outer')

In [284]:
df_king

,DateTime,intra_price(円/kWh),spot_price(円/kWh),gap_price[intra-spot](円/kWh),intra_volume（MWh/h）,spot_volume(kWh),九電太陽想定値(kWh),九電太陽実績値(kWh),九電風力想定値(kWh),九電風力実績値(kWh),...,東電風力想定(kWh),東電風力実績(kWh),関電太陽光想定(kWh),関電太陽光実績(kWh),関電風力想定(kWh),関電風力実績(kWh),中国太陽光想定(kWh),中国風力想定(kWh),中国太陽光実績(kWh),中国風力実績(kWh)
0,2018-04-01 00:00:00,7.62,7.30,0.32,58.8,6098500,0.0,0.0,5889.905,8909.1,...,41386,50919,0,0,18118,21411,0,39041,0,38993
1,2018-04-01 00:30:00,7.52,6.93,0.59,54.0,6225500,0.0,0.0,5642.430,8909.1,...,41615,50733,0,0,18118,22064,0,41624,0,40253
2,2018-04-01 01:00:00,7.50,6.78,0.72,57.3,6356500,0.0,0.0,5642.430,6929.3,...,41793,51667,0,0,18118,21297,0,43205,0,37977
3,2018-04-01 01:30:00,7.56,6.84,0.72,63.0,6439500,0.0,0.0,5444.450,5939.4,...,42116,57049,0,0,18118,19419,0,43721,0,39237
4,2018-04-01 02:00:00,6.89,6.92,-0.03,61.6,6439000,0.0,0.0,5295.965,5939.4,...,42244,55349,0,0,18118,14621,0,43721,0,37666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79016,2022-09-30 21:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,11577,0,2465
79017,2022-09-30 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,10829,0,4302
79018,2022-09-30 22:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,13621,0,2447
79019,2022-09-30 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,14413,0,1386


In [231]:
columns = ['DATE','TIME','中国太陽光想定(kWh)','中国風力想定(kWh)','中国太陽光実績(kWh)','中国風力実績(kWh)']
df_chugoku_plan_2020_ = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/fit_tokurei1.csv',encoding='shift-jis',names=columns)[3:]

In [243]:
df_chugoku_plan_2018 = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/kako-2018.csv',encoding='shift-jis',names=columns)[3:].dropna()
df_chugoku_plan_2019 = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/kako-2019.csv',encoding='shift-jis',names=columns)[3:].dropna()


In [274]:
def Get_Chugoku_Plan():
    columns = ['DATE','TIME','中国太陽光想定(kWh)','中国風力想定(kWh)','中国太陽光実績(kWh)','中国風力実績(kWh)']
    # ファイルを全て読み込み
    df_chugoku_plan_2020_ = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/fit_tokurei1.csv',encoding='shift-jis',names=columns)[3:]
    df_chugoku_plan_2018 = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/kako-2018.csv',encoding='shift-jis',names=columns)[3:].dropna()
    df_chugoku_plan_2019 = pd.read_csv('../../卒論関連書類/データ/発電予測値/中国電力/kako-2019.csv',encoding='shift-jis',names=columns)[3:].dropna()
    # データを結合
    df_total_chugoku_plan =pd.concat([df_chugoku_plan_2018,df_chugoku_plan_2019,df_chugoku_plan_2020_],axis=0).reset_index(drop=True)
    # //timestampを作成
    datetime_list = []
    date_list, time_list = df_total_chugoku_plan['DATE'].values, df_total_chugoku_plan['TIME'].values

    for i in range(len(date_list)):
        str_datetime = date_list[i]+'/'+time_list[i]
        datetime_list.append(dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M'))
    # //
    df_total_chugoku_plan['DateTime'] = datetime_list
    df_total_chugoku_plan = df_total_chugoku_plan.drop(columns=['DATE','TIME'])
    # 左側にDateTimeを持ってきた
    df_total_chugoku_plan =df_total_chugoku_plan.reindex(columns=['DateTime','中国太陽光想定(kWh)','中国風力想定(kWh)','中国太陽光実績(kWh)','中国風力実績(kWh)'])
    
    return(df_total_chugoku_plan)



In [270]:
df_total_chugoku_plan =pd.concat([df_chugoku_plan_2018,df_chugoku_plan_2019,df_chugoku_plan_2020_],axis=0).reset_index(drop=True)
# //timestampを作成
datetime_list = []
date_list, time_list = df_total_chugoku_plan['DATE'].values, df_total_chugoku_plan['TIME'].values

for i in range(len(date_list)):
    str_datetime = date_list[i]+'/'+time_list[i]
    datetime_list.append(dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M'))
# //
df_total_chugoku_plan['DateTime'] = datetime_list
df_total_chugoku_plan = df_total_chugoku_plan.drop(columns=['DATE','TIME'])
# 左側にDateTimeを持ってきた
df_total_chugoku_plan =df_total_chugoku_plan.reindex(columns=['DateTime','中国太陽光想定(kWh)','中国風力想定(kWh)','中国太陽光実績(kWh)','中国風力実績(kWh)'])

In [273]:
df_total_chugoku_plan =df_total_chugoku_plan.reindex(columns=['DateTime','中国太陽光想定(kWh)','中国風力想定(kWh)','中国太陽光実績(kWh)','中国風力実績(kWh)'])

In [278]:
df_chugoku_plan

,DateTime,中国太陽光想定(kWh),中国風力想定(kWh),中国太陽光実績(kWh),中国風力実績(kWh)
0,2018-04-01 00:00:00,0,39041,0,38993
1,2018-04-01 00:30:00,0,41624,0,40253
2,2018-04-01 01:00:00,0,43205,0,37977
3,2018-04-01 01:30:00,0,43721,0,39237
4,2018-04-01 02:00:00,0,43721,0,37666
...,...,...,...,...,...
78907,2022-09-30 21:30:00,0,11577,0,2465
78908,2022-09-30 22:00:00,0,10829,0,4302
78909,2022-09-30 22:30:00,0,13621,0,2447
78910,2022-09-30 23:00:00,0,14413,0,1386


In [257]:
datetime_list[-1]

datetime.datetime(2022, 9, 30, 23, 30)

In [256]:
df_total_chuden_plan

,DATE,TIME,中国太陽光想定(kWh),中国風力想定(kWh),中国太陽光実績(kWh),中国風力実績(kWh)
0,2018/4/1,0:00,0,39041,0,38993
1,2018/4/1,0:30,0,41624,0,40253
2,2018/4/1,1:00,0,43205,0,37977
3,2018/4/1,1:30,0,43721,0,39237
4,2018/4/1,2:00,0,43721,0,37666
...,...,...,...,...,...,...
78907,2022/9/30,21:30,0,11577,0,2465
78908,2022/9/30,22:00,0,10829,0,4302
78909,2022/9/30,22:30,0,13621,0,2447
78910,2022/9/30,23:00,0,14413,0,1386


In [232]:
df_chugoku_plan_2020_

,DATE,TIME,中国太陽光想定(kWh),中国風力想定(kWh),中国太陽光実績(kWh),中国風力実績(kWh)
3,2020/4/1,0:00,0,8227,0,13548
4,2020/4/1,0:30,0,9198,0,14139
5,2020/4/1,1:00,0,10699,0,10500
6,2020/4/1,1:30,0,11200,0,11035
7,2020/4/1,2:00,0,7163,0,6838
...,...,...,...,...,...,...
43822,2022/9/30,21:30,0,11577,0,2465
43823,2022/9/30,22:00,0,10829,0,4302
43824,2022/9/30,22:30,0,13621,0,2447
43825,2022/9/30,23:00,0,14413,0,1386


In [241]:
df_chugoku_plan_2018

,DATE,TIME,中国太陽光想定(kWh),中国風力想定(kWh),中国太陽光実績(kWh),中国風力実績(kWh)
3,2018/4/1,0:00,0,39041,0,38993
4,2018/4/1,0:30,0,41624,0,40253
5,2018/4/1,1:00,0,43205,0,37977
6,2018/4/1,1:30,0,43721,0,39237
7,2018/4/1,2:00,0,43721,0,37666
...,...,...,...,...,...,...
17518,2019/3/31,21:30,0,24252,0,26744
17519,2019/3/31,22:00,0,25785,0,30633
17520,2019/3/31,22:30,0,27307,0,24535
17521,2019/3/31,23:00,0,31350,0,36008


In [242]:
df_chugoku_plan_2020_.dropna()

,DATE,TIME,中国太陽光想定(kWh),中国風力想定(kWh),中国太陽光実績(kWh),中国風力実績(kWh)
3,2020/4/1,0:00,0,8227,0,13548
4,2020/4/1,0:30,0,9198,0,14139
5,2020/4/1,1:00,0,10699,0,10500
6,2020/4/1,1:30,0,11200,0,11035
7,2020/4/1,2:00,0,7163,0,6838
...,...,...,...,...,...,...
43822,2022/9/30,21:30,0,11577,0,2465
43823,2022/9/30,22:00,0,10829,0,4302
43824,2022/9/30,22:30,0,13621,0,2447
43825,2022/9/30,23:00,0,14413,0,1386


In [185]:
# 関電
year = 2018
df_kanden_plan =pd.read_csv(f'../../卒論関連書類/データ/発電予測値/関西電力/{year}fit1_soutei_jisseki.csv',encoding='shift-jis')

In [197]:
str_datetime_list = df_kanden_plan['DATE_TIME'].values

In [196]:
datetime_list[0

'2018/4/1 0:00'

In [206]:
str_datetime_list = df_kanden_plan['DATE_TIME'].values
datetime_list=[]
for str_datetime in str_datetime_list:
    datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d %H:%M')
    datetime_list.append(datetime)

In [207]:
datetime_list

[datetime.datetime(2018, 4, 1, 0, 0),
 datetime.datetime(2018, 4, 1, 0, 30),
 datetime.datetime(2018, 4, 1, 1, 0),
 datetime.datetime(2018, 4, 1, 1, 30),
 datetime.datetime(2018, 4, 1, 2, 0),
 datetime.datetime(2018, 4, 1, 2, 30),
 datetime.datetime(2018, 4, 1, 3, 0),
 datetime.datetime(2018, 4, 1, 3, 30),
 datetime.datetime(2018, 4, 1, 4, 0),
 datetime.datetime(2018, 4, 1, 4, 30),
 datetime.datetime(2018, 4, 1, 5, 0),
 datetime.datetime(2018, 4, 1, 5, 30),
 datetime.datetime(2018, 4, 1, 6, 0),
 datetime.datetime(2018, 4, 1, 6, 30),
 datetime.datetime(2018, 4, 1, 7, 0),
 datetime.datetime(2018, 4, 1, 7, 30),
 datetime.datetime(2018, 4, 1, 8, 0),
 datetime.datetime(2018, 4, 1, 8, 30),
 datetime.datetime(2018, 4, 1, 9, 0),
 datetime.datetime(2018, 4, 1, 9, 30),
 datetime.datetime(2018, 4, 1, 10, 0),
 datetime.datetime(2018, 4, 1, 10, 30),
 datetime.datetime(2018, 4, 1, 11, 0),
 datetime.datetime(2018, 4, 1, 11, 30),
 datetime.datetime(2018, 4, 1, 12, 0),
 datetime.datetime(2018, 4, 1, 12

In [216]:
def Get_Kanden_Plan():

    df_kanden_plan_king = pd.DataFrame(columns=['DateTime','関電太陽光想定(kWh)','関電太陽光実績(kWh)','関電風力想定(kWh)','関電風力実績(kWh)'])

    for year in range(2018,2022):
        df_kanden_plan =pd.read_csv(f'../../卒論関連書類/データ/発電予測値/関西電力/{year}fit1_soutei_jisseki.csv',encoding='shift-jis')
        df_kanden_plan_total = pd.DataFrame(columns=['DateTime','関電太陽光想定(kWh)','関電太陽光実績(kWh)','関電風力想定(kWh)','関電風力実績(kWh)'])

        str_datetime_list = df_kanden_plan['DATE_TIME'].values
        datetime_list=[]
        for str_datetime in str_datetime_list:
            datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d %H:%M')
            datetime_list.append(datetime)
        # 値の代入
        df_kanden_plan_total['DateTime'] = datetime_list
        df_kanden_plan_total['関電太陽光想定(kWh)'] = df_kanden_plan['FIT1_太陽光想定値［kWh］']
        df_kanden_plan_total['関電太陽光実績(kWh)'] = df_kanden_plan['FIT1_太陽光実績値［kWh］']
        df_kanden_plan_total['関電風力想定(kWh)'] = df_kanden_plan['FIT1_風力想定値［kWh］']
        df_kanden_plan_total['関電風力実績(kWh)'] = df_kanden_plan['FIT1_風力実績値［kWh］']

        # 結合
        df_kanden_plan_king = pd.concat([df_kanden_plan_king,df_kanden_plan_total],axis=0)
    return(df_kanden_plan_king)

In [208]:
df_kanden_plan

,DATE_TIME,FIT1_太陽光想定値［kWh］,FIT1_太陽光実績値［kWh］,FIT1_風力想定値［kWh］,FIT1_風力実績値［kWh］
0,2018/4/1 0:00,0,0,18118,21411
1,2018/4/1 0:30,0,0,18118,22064
2,2018/4/1 1:00,0,0,18118,21297
3,2018/4/1 1:30,0,0,18118,19419
4,2018/4/1 2:00,0,0,18118,14621
...,...,...,...,...,...
17515,2019/3/31 21:30,0,0,20912,20591
17516,2019/3/31 22:00,0,0,20912,21016
17517,2019/3/31 22:30,0,0,20912,22737
17518,2019/3/31 23:00,0,0,20912,22597


In [222]:
Get_Kanden_Plan()

,DateTime,関電太陽光想定(kWh),関電太陽光実績(kWh),関電風力想定(kWh),関電風力実績(kWh)
0,2018-04-01 00:00:00,0,0,18118,21411
1,2018-04-01 00:30:00,0,0,18118,22064
2,2018-04-01 01:00:00,0,0,18118,21297
3,2018-04-01 01:30:00,0,0,18118,19419
4,2018-04-01 02:00:00,0,0,18118,14621
...,...,...,...,...,...
17515,2022-03-31 21:30:00,0,0,"17,549","45,310"
17516,2022-03-31 22:00:00,0,0,"18,966","52,083"
17517,2022-03-31 22:30:00,0,0,"20,660","60,271"
17518,2022-03-31 23:00:00,0,0,"22,572","61,016"


In [214]:
df_kanden_plan.columns

Index(['DATE_TIME', 'FIT1_太陽光想定値［kWh］', 'FIT1_太陽光実績値［kWh］', 'FIT1_風力想定値［kWh］',
       'FIT1_風力実績値［kWh］'],
      dtype='object')

In [181]:
df_toden_plan

,DateTime,東電太陽光想定(kWh),東電太陽光実績(kWh),東電風力想定(kWh),東電風力実績(kWh)
0,2018-04-01 00:00:00,0,0,41386,50919
1,2018-04-01 00:30:00,0,0,41615,50733
2,2018-04-01 01:00:00,0,0,41793,51667
3,2018-04-01 01:30:00,0,0,42116,57049
4,2018-04-01 02:00:00,0,0,42244,55349
...,...,...,...,...,...
17515,2022-03-31 21:30:00,0,0,79325,67314
17516,2022-03-31 22:00:00,0,0,81166,72550
17517,2022-03-31 22:30:00,0,0,85104,71513
17518,2022-03-31 23:00:00,0,0,89412,66491


In [137]:
df_king

,DateTime,intra_price(円/kWh),spot_price(円/kWh),gap_price[intra-spot](円/kWh),intra_volume（MWh/h）,spot_volume(kWh),九電太陽想定値(kWh),九電太陽実績値(kWh),九電風力想定値(kWh),九電風力実績値(kWh)
0,2018-04-01 00:00:00,7.62,7.30,0.32,58.8,6098500,0.0,0.0,5889.905000,8909.100000
1,2018-04-01 00:30:00,7.52,6.93,0.59,54.0,6225500,0.0,0.0,5642.430000,8909.100000
2,2018-04-01 01:00:00,7.50,6.78,0.72,57.3,6356500,0.0,0.0,5642.430000,6929.300000
3,2018-04-01 01:30:00,7.56,6.84,0.72,63.0,6439500,0.0,0.0,5444.450000,5939.400000
4,2018-04-01 02:00:00,6.89,6.92,-0.03,61.6,6439000,0.0,0.0,5295.965000,5939.400000
...,...,...,...,...,...,...,...,...,...,...
70123,2022-03-31 21:30:00,22.13,23.40,-1.27,1116.4,17418300,0.0,0.0,75589.876734,80915.101876
70124,2022-03-31 22:00:00,24.66,28.34,-3.68,995.2,16727350,0.0,0.0,78357.541014,82754.081464
70125,2022-03-31 22:30:00,17.31,23.96,-6.65,797.1,16534600,0.0,0.0,76204.555661,85512.550846
70126,2022-03-31 23:00:00,15.81,22.80,-6.99,649.0,16452950,0.0,0.0,76632.118415,89190.510023


In [135]:
df_jepx_total

,DateTime,intra_price(円/kWh),spot_price(円/kWh),gap_price[intra-spot](円/kWh),intra_volume（MWh/h）,spot_volume(kWh)
0,2018-04-01 00:00:00,7.62,7.30,0.32,58.8,6098500
1,2018-04-01 00:30:00,7.52,6.93,0.59,54.0,6225500
2,2018-04-01 01:00:00,7.50,6.78,0.72,57.3,6356500
3,2018-04-01 01:30:00,7.56,6.84,0.72,63.0,6439500
4,2018-04-01 02:00:00,6.89,6.92,-0.03,61.6,6439000
...,...,...,...,...,...,...
70123,2022-03-31 21:30:00,22.13,23.40,-1.27,1116.4,17418300
70124,2022-03-31 22:00:00,24.66,28.34,-3.68,995.2,16727350
70125,2022-03-31 22:30:00,17.31,23.96,-6.65,797.1,16534600
70126,2022-03-31 23:00:00,15.81,22.80,-6.99,649.0,16452950


In [ ]:
# 東電

In [141]:
df_toden_plan = pd.read_csv('../../卒論関連書類/データ/発電予測値/東京電力/fit-2018.csv',encoding='shift-jis')

In [173]:
def Get_Toden_Plan():

    df_toden_plan_king = pd.DataFrame(columns=['DateTime','東電太陽光想定(kWh)','東電太陽光実績(kWh)','東電風力想定(kWh)','東電風力実績(kWh)'])

    for year in range(2018,2022):
        df_toden_plan = pd.read_csv(f'../../卒論関連書類/データ/発電予測値/東京電力/fit-{year}.csv',encoding='shift-jis')
        df_toden_plan_total = pd.DataFrame(columns=['DateTime','東電太陽光想定(kWh)','東電太陽光実績(kWh)','東電風力想定(kWh)','東電風力実績(kWh)'])

        # timestampの列を作成
        date_list =  df_toden_plan['DATE'].values
        time_list = df_toden_plan['TIME'].values
        str_datetime_list = date_list+'/'+time_list

        datetime_list = []
        for str_datetime in str_datetime_list:
            datetime = dt.datetime.strptime(str_datetime,'%Y/%m/%d/%H:%M')
            datetime_list.append(datetime)
        # 値の代入
        df_toden_plan_total['DateTime'] = datetime_list
        df_toden_plan_total['東電太陽光想定(kWh)'] = df_toden_plan['太陽光想定(kWh)']
        df_toden_plan_total['東電太陽光実績(kWh)'] = df_toden_plan['太陽光実績(kWh)']
        df_toden_plan_total['東電風力想定(kWh)'] = df_toden_plan['風力想定(kWh)']
        df_toden_plan_total['東電風力実績(kWh)'] = df_toden_plan['風力実績(kWh)']

        # 結合
        df_toden_plan_king = pd.concat([df_toden_plan_king,df_toden_plan_total],axis=0)
        return(df_toden_plan_king)




        

In [171]:
df_toden_plan_king

,DateTime,東電太陽光想定(kWh),東電太陽光実績(kWh),東電風力想定(kWh),東電風力実績(kWh)
0,2018-04-01 00:00:00,0,0,41386,50919
1,2018-04-01 00:30:00,0,0,41615,50733
2,2018-04-01 01:00:00,0,0,41793,51667
3,2018-04-01 01:30:00,0,0,42116,57049
4,2018-04-01 02:00:00,0,0,42244,55349
...,...,...,...,...,...
17515,2022-03-31 21:30:00,0,0,79325,67314
17516,2022-03-31 22:00:00,0,0,81166,72550
17517,2022-03-31 22:30:00,0,0,85104,71513
17518,2022-03-31 23:00:00,0,0,89412,66491


In [172]:
df_toden_plan_king.dropna

<bound method DataFrame.dropna of                  DateTime 東電太陽光想定(kWh) 東電太陽光実績(kWh) 東電風力想定(kWh) 東電風力実績(kWh)
0     2018-04-01 00:00:00            0            0       41386       50919
1     2018-04-01 00:30:00            0            0       41615       50733
2     2018-04-01 01:00:00            0            0       41793       51667
3     2018-04-01 01:30:00            0            0       42116       57049
4     2018-04-01 02:00:00            0            0       42244       55349
...                   ...          ...          ...         ...         ...
17515 2022-03-31 21:30:00            0            0       79325       67314
17516 2022-03-31 22:00:00            0            0       81166       72550
17517 2022-03-31 22:30:00            0            0       85104       71513
17518 2022-03-31 23:00:00            0            0       89412       66491
17519 2022-03-31 23:30:00            0            0       93876       75255

[70128 rows x 5 columns]>

In [148]:
df_toden_plan_total = pd.DataFrame(columns=['DateTime','東電太陽光想定(kWh)','東電太陽光実績(kWh)','東電風力想定(kWh)','東電風力実績(kWh)'])

In [147]:
df_toden_plan

,DATE,TIME,太陽光想定(kWh),太陽光実績(kWh),風力想定(kWh),風力実績(kWh)
0,2018/4/1,0:00,0,0,41386,50919
1,2018/4/1,0:30,0,0,41615,50733
2,2018/4/1,1:00,0,0,41793,51667
3,2018/4/1,1:30,0,0,42116,57049
4,2018/4/1,2:00,0,0,42244,55349
...,...,...,...,...,...,...
17515,2019/3/31,21:30,0,0,65272,62975
17516,2019/3/31,22:00,0,0,63538,66773
17517,2019/3/31,22:30,0,0,61720,65595
17518,2019/3/31,23:00,0,0,60018,61448


In [ ]:
for i in range(len(df_toden_plan)):
    str_date = 

In [123]:
df_kyuden_plan

,DateTime,九電太陽想定値(kWh),九電太陽実績値(kWh),九電風力想定値(kWh),九電風力実績値(kWh)
0,2018-04-01 00:00:00,0.0,0.0,5889.905000,8909.100000
1,2018-04-01 00:30:00,0.0,0.0,5642.430000,8909.100000
2,2018-04-01 01:00:00,0.0,0.0,5642.430000,6929.300000
3,2018-04-01 01:30:00,0.0,0.0,5444.450000,5939.400000
4,2018-04-01 02:00:00,0.0,0.0,5295.965000,5939.400000
...,...,...,...,...,...
70123,2022-03-31 21:30:00,0.0,0.0,75589.876734,80915.101876
70124,2022-03-31 22:00:00,0.0,0.0,78357.541014,82754.081464
70125,2022-03-31 22:30:00,0.0,0.0,76204.555661,85512.550846
70126,2022-03-31 23:00:00,0.0,0.0,76632.118415,89190.510023


In [127]:
df_kyuden_plan

,DateTime,九電太陽想定値(kWh),九電太陽実績値(kWh),九電風力想定値(kWh),九電風力実績値(kWh)
0,2018-04-01 00:00:00,0.0,0.0,5889.905000,8909.100000
1,2018-04-01 00:30:00,0.0,0.0,5642.430000,8909.100000
2,2018-04-01 01:00:00,0.0,0.0,5642.430000,6929.300000
3,2018-04-01 01:30:00,0.0,0.0,5444.450000,5939.400000
4,2018-04-01 02:00:00,0.0,0.0,5295.965000,5939.400000
...,...,...,...,...,...
70123,2022-03-31 21:30:00,0.0,0.0,75589.876734,80915.101876
70124,2022-03-31 22:00:00,0.0,0.0,78357.541014,82754.081464
70125,2022-03-31 22:30:00,0.0,0.0,76204.555661,85512.550846
70126,2022-03-31 23:00:00,0.0,0.0,76632.118415,89190.510023


In [130]:
df_kyuden_plan[df_kyuden_plan['九電風力想定値(kWh)'].isnull()]
# 確認済み

,DateTime,九電太陽想定値(kWh),九電太陽実績値(kWh),九電風力想定値(kWh),九電風力実績値(kWh)
29576,2019-12-08 04:00:00,0.0,0.0,NaN,43312.5


In [109]:
time_code =df_spot['時刻コード'].values
time_code = (time_code-1)*30
hour_list,min_list = time_code//60, time_code%60

In [108]:
time_code[0]%60

30

In [110]:
hour_list


array([ 0,  0,  1, ..., 22, 23, 23])

In [111]:
min_list

array([ 0, 30,  0, ..., 30,  0, 30])

In [112]:
dt.datetime.strptime(f'2019/03/21/{hour_list[0]}/{min_list[1]}','%Y/%m/%d/%H/%M')

datetime.datetime(2019, 3, 21, 0, 30)

In [113]:
date_list=[]

spot_dates = df_spot['年月日'].values
# 時刻コードを時間に直す(0時0分ー23時30分)
time_code =df_spot['時刻コード'].values
time_code = (time_code-1)*30
hour_list,min_list = time_code//60, time_code%60

for i in range(len(spot_dates)):
    str_date = f'{spot_dates[i]}/{hour_list[i]}/{min_list[i]}'

    date = dt.datetime.strptime(str_date,'%Y/%m/%d/%H/%M')
    date_list.append(date)

In [114]:
date_list

[datetime.datetime(2021, 4, 1, 0, 0),
 datetime.datetime(2021, 4, 1, 0, 30),
 datetime.datetime(2021, 4, 1, 1, 0),
 datetime.datetime(2021, 4, 1, 1, 30),
 datetime.datetime(2021, 4, 1, 2, 0),
 datetime.datetime(2021, 4, 1, 2, 30),
 datetime.datetime(2021, 4, 1, 3, 0),
 datetime.datetime(2021, 4, 1, 3, 30),
 datetime.datetime(2021, 4, 1, 4, 0),
 datetime.datetime(2021, 4, 1, 4, 30),
 datetime.datetime(2021, 4, 1, 5, 0),
 datetime.datetime(2021, 4, 1, 5, 30),
 datetime.datetime(2021, 4, 1, 6, 0),
 datetime.datetime(2021, 4, 1, 6, 30),
 datetime.datetime(2021, 4, 1, 7, 0),
 datetime.datetime(2021, 4, 1, 7, 30),
 datetime.datetime(2021, 4, 1, 8, 0),
 datetime.datetime(2021, 4, 1, 8, 30),
 datetime.datetime(2021, 4, 1, 9, 0),
 datetime.datetime(2021, 4, 1, 9, 30),
 datetime.datetime(2021, 4, 1, 10, 0),
 datetime.datetime(2021, 4, 1, 10, 30),
 datetime.datetime(2021, 4, 1, 11, 0),
 datetime.datetime(2021, 4, 1, 11, 30),
 datetime.datetime(2021, 4, 1, 12, 0),
 datetime.datetime(2021, 4, 1, 12